### 並進ベクトルの抽出

並進して重なるような局所構造の対をさがし、それらをつなぐベクトルを列挙する。

過去にそのようなプログラムを書いた覚えがある。


In [15]:
import numpy as np
from read_gro3 import read_gro
import networkx as nx
import pairlist as pl
import yaplotlib as yap

with open("/Users/matto/work/master_project/250K3000bar/300-last.gro") as f:
# with open("300-last.gro") as f:
    for gro in read_gro(f):
        break

# cell matrix
cellmat = gro["cell"]

# inverse of the cell matrix
celli = np.linalg.inv(cellmat)

# pickup positions of carbon atoms, in the fractional coordinate
atomPositions = gro["position"]
carbons = atomPositions[gro["atom"] == "C"] @ celli


def smatcher(
    vertices: np.ndarray, cellMatrix: np.ndarray, templateRadius: float, allowance: float
):
    """Slide and match the local atomic arrangements

    Args:
        vertices (np.ndarray): atomic positions (fractional)
        cellMatrix (np.ndarray): cell matrix = bundle of cell vectors
        templateRadius (float): cropping radius of the template
        allowance (float): allowed displacements between the template and target

    Yields:
        _type_: _description_
    """
    f = open("log.yap", "w")

    f.write(yap.Line(np.zeros(3), cellMatrix[0]))
    f.write(yap.Line(np.zeros(3), cellMatrix[1]))
    f.write(yap.Line(np.zeros(3), cellMatrix[2]))
    f.write(yap.Color(3))

    # make the adjacency matrix within radius
    g = nx.Graph(
        [
            (i, j)
            for i, j in pl.pairs_iter(vertices, templateRadius, cellMatrix, distance=False)
        ]
    )

    # 各原子の周囲にある原子の座標
    neighbors = []
    for v, vertex in enumerate(vertices):
        nei = g[v]
        d = vertices[nei] - vertex
        # PBC
        d -= np.floor(d + 0.5)
        neighbors.append(d)

    # 照合作業。pythonではつらい。あまり遠いペアは今は照合しない。
    for i, j in pl.pairs_iter(vertices, templateRadius * 2, cellMatrix, distance=False):
        nei = neighbors[i] @ cellMatrix
        nej = neighbors[j] @ cellMatrix
        count = 0
        # matching without PBC
        for a, b, d in pl.pairs_iter(nei, allowance, pos2=nej):
            count += 1
        # テンプレートの原子数の半分以上が、ターゲットの周囲の原子に近いと判定されたら、
        if count > len(nei) // 2:
            d = vertices[j] - vertices[i]
            d -= np.floor(d + 0.5)
            yield i, j, templateRadius, allowance, count, d @ cellMatrix
            print(
                yap.Line(vertices[i] @ cellMatrix, (vertices[i] + d) @ cellMatrix),
                end="",
                file=f,
            )
            f.flush()


# テンプレートの大きさ。
templateRadius = 0.5  # nm

# 照合時のずれ長さの許容範囲
allowance = 0.1


vecs = []
target = []
for smatch in smatcher(carbons, cellmat, templateRadius, allowance):
    # i,j : pair of similar arrangement
    # radius, matchradius: conditions
    # count: number of matched atoms
    # vec: relative vector
    i, j, templateRadius, allowance, count, vec = smatch
    vecs.append(vec)
    target.append(i)

vecs = np.array(vecs)
zpos = carbons[target, 2]

In [16]:
# z相対座標が0.2以下の場合だけを

import plotly.graph_objects as go
import numpy as np

fig = go.Figure()

fig.add_trace(
    go.Scatter3d(
        x=vecs[:, 0],
        y=vecs[:, 1],
        z=vecs[:, 2],
        # x=vecs[zpos<0.5, 0],
        # y=vecs[zpos<0.5, 1],
        # z=vecs[zpos<0.5, 2],
        mode="markers",
        marker=dict(size=2, color=zpos[:]),
    )
)

fig.update_layout(
    title="local arrangement",
    autosize=True,
    width=800,
    height=800,
    margin=dict(l=65, r=50, b=65, t=90),
)

fig.show()

FCCに見える。